<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

<b>Pascal P., March 2020</b>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
# !wget -Nc  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [2]:
# !unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need.  
   Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [4]:
num_classes  = 2
image_resize = 224

batch_size_train = 100
batch_size_val   = 100

In [5]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

train_generator = data_generator.flow_from_directory(
  'concrete_data_week4/train',
  target_size=(image_resize, image_resize),
  batch_size=batch_size_train,
  class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [6]:
valid_generator = data_generator.flow_from_directory(
  'concrete_data_week4/valid',
  target_size=(image_resize, image_resize),
  batch_size=batch_size_val,
  class_mode='categorical' 
)

Found 9501 images belonging to 2 classes.


In [7]:
model_vgg16 = Sequential()

model_vgg16.add(
  vgg16.VGG16(include_top=False, pooling='avg', weights='imagenet')
)
model_vgg16.add(Dense(num_classes, activation='softmax'))

W0330 19:36:57.159770 140619480467264 deprecation.py:506] From /home/pascal/Projects/ML_DL/anaconda3/envs/tensorflow_keras_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0330 19:36:57.176990 140619480467264 module_wrapper.py:139] From /home/pascal/Projects/ML_DL/anaconda3/envs/tensorflow_keras_gpuenv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [8]:
model_vgg16.layers[0].trainable = False

In [9]:
model_vgg16.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [10]:
model_vgg16.compile(optimizer='adam', 
                    loss='categorical_crossentropy', 
                    metrics=['accuracy'])

In [11]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(valid_generator)

num_epochs = 2

In [12]:
fit_history = model_vgg16.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

W0330 19:36:58.560511 140619480467264 module_wrapper.py:139] From /home/pascal/Projects/ML_DL/anaconda3/envs/tensorflow_keras_gpuenv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/2
301/301 [==============================] - 215s 715ms/step - loss: 0.1306 - accuracy: 0.9528 - val_loss: 0.0273 - val_accuracy: 0.9932
Epoch 2/2
301/301 [==============================] - 212s 703ms/step - loss: 0.0246 - accuracy: 0.9950 - val_loss: 1.0729e-05 - val_accuracy: 0.9955


In [13]:
model_vgg16.save('classifier_vgg16_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set.  
   For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [14]:
# from keras.applications import resnet50
from keras.models import load_model

file_model = '../w3-keras/classifier_resnet_model.h5'

model_resnet50 = load_model(file_model)

/home/pascal/Projects/ML_DL/anaconda3/envs/tensorflow_keras_gpuenv/lib/python3.7/site-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [15]:
test_data_generator = ImageDataGenerator()


In [16]:
test_generator = test_data_generator.flow_from_directory(
  'concrete_data_week4/test',
  target_size=(image_resize, image_resize),
  shuffle=False)

Found 500 images belonging to 2 classes.


In [17]:
perf_vgg_16 = model_vgg16.evaluate_generator(test_generator, 
                                             verbose=0)

In [18]:
perf_resnet_50 = model_resnet50.evaluate_generator(test_generator, 
                                                   verbose=0)

In [36]:
# ['loss', 'accuracy']
print("perf.    vgg_16: ", perf_vgg_16, "\nperf. resnet_50: ", perf_resnet_50)

perf.    vgg_16:  [0.4253148138523102, 0.9160000085830688] 
perf. resnet_50:  [3.126983165740967, 0.656000018119812]


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [22]:
# positive class == sample with cracks ==> encoded 1 == [0, 1]
# negative class == sample without cracks ==> encoded 0 == [1, 0]

In [23]:
pred_vgg16 = model_vgg16.predict_generator(test_generator, verbose=0)

In [24]:
pred_vgg16.shape

(500, 2)

In [25]:
pred_vgg16[400:410]

array([[8.6281407e-06, 9.9999142e-01],
       [3.2362002e-03, 9.9676383e-01],
       [1.3653945e-02, 9.8634607e-01],
       [4.3093578e-06, 9.9999571e-01],
       [4.5616433e-01, 5.4383564e-01],
       [4.7432718e-06, 9.9999523e-01],
       [1.1065661e-01, 8.8934338e-01],
       [9.8099077e-01, 1.9009273e-02],
       [2.8042671e-01, 7.1957332e-01],
       [2.3688973e-08, 1.0000000e+00]], dtype=float32)

In [26]:
# x, y = pred_vgg16[475:500][0]
# array([0.00150147, 0.99849856], dtype=float32)

ix = 400
start = ix
for v1, v2 in pred_vgg16[start:start + 10]:
  if v1 > v2: 
      print("- Sample ", ix, " is Negative")
  else: 
      print("+ Sample ", ix, " is Positive")
  ix += 1

+ Sample  400  is Positive
+ Sample  401  is Positive
+ Sample  402  is Positive
+ Sample  403  is Positive
+ Sample  404  is Positive
+ Sample  405  is Positive
+ Sample  406  is Positive
- Sample  407  is Negative
+ Sample  408  is Positive
+ Sample  409  is Positive


In [27]:
pred_resnet50 = model_resnet50.predict_generator(test_generator, verbose=0)

In [28]:
pred_resnet50.shape

(500, 2)

In [29]:
ix = 400
start = ix
for v1, v2 in pred_resnet50[start:start + 10]:
  if v1 > v2: 
      print("- Sample ", ix, " is Negative")
  else: 
      print("+ Sample ", ix, " is Positive")
  ix += 1

- Sample  400  is Negative
- Sample  401  is Negative
+ Sample  402  is Positive
+ Sample  403  is Positive
- Sample  404  is Negative
+ Sample  405  is Positive
- Sample  406  is Negative
- Sample  407  is Negative
- Sample  408  is Negative
- Sample  409  is Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).